In [1]:
import cv2
#import helper1
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random as rng
import os
import glob 
import pandas as pd
%matplotlib inline

In [2]:
def canny(selected_image):
    #selected_image = cv2.resize(selected_image, (0, 0), fx = 0.9, fy = 0.9) 
    image = cv2.cvtColor(selected_image, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    #plt.imshow(gray)
    cv2.imwrite('B1.jpg',gray)

    retval, binary = cv2.threshold(gray,90, 255,0)
    #plt.imshow(binary, cmap='gray')
    cv2.imwrite('B2.jpg',binary)

    edged = cv2.Canny(gray ,15,180) 
    #plt.imshow(edged, cmap='gray')
    #cv2.imwrite('B3.jpg',edged)
    return(binary,edged)

In [3]:
def contourrr(image,binary):
    #retval, contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours, hierarchy = cv2.findContours(binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    # Draw all contours on a copy of the original image
    contours_image = np.copy(image)
    print(len(contours))

    contours_image = cv2.drawContours(contours_image, contours, -1, (0,0,255), 3)

    #plt.imshow(contours_image)
    #cv2.imwrite('B5.jpg',contours_image)
    return(contours)

In [4]:
def roi(image,im_binary,contours,ROI_number):
    global roi1,roi2
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        #print(cv2.contourArea(c))
        z = cv2.contourArea(c) 
        if(z>300000 and z<490000):
            print('--')
            print(cv2.contourArea(c))
            img = cv2.rectangle(image,(x-30,y-30),(x+w+30,y+h+30),(0,255,0),3)
            roi1 = image[y-30:y+h+30,x-30:x+w+30]
            #cv2.imwrite('B1000.jpg',roi1)
        elif(z>80000 and z<300000):
            print('--')
            print(cv2.contourArea(c))
            img = cv2.rectangle(image,(x-10,y-20),(x+w+20,y+h+20),(0,255,0),3)
            roi2 = im_binary[y-20:y+h+20,x-20:x+w+20]
            #cv2.imwrite('B1011.jpg',roi2)
    
    #cv2.imwrite("0,1/Color_image"+str(ROI_number)+".jpg",roi1)
    
    #print('*')
    #print(ROI_number)




    #plt.imshow(img,cmap='gray')
    #cv2.imwrite('B6.jpg',img)
    
    return(roi1,roi2,img)


In [5]:
train_dir = "0,1/train/"
test_dir = "0,1/test/"

def loadtrain_dataset(image_dir):
    ROI_number = 1
    train_images = []
    train_labels = [] 
    # Populate this empty image list
    image_types = ["Defected","Ok"]
    
    # Iterate through each color folder
    for im_type in image_types:
        
        # Iterate through each image file in each image_type folder
        # glob reads in any image with the extension "image_dir/im_type/*"
        for file in glob.glob(os.path.join(image_dir, im_type, "*")):
            
            # Read in the image
            im = cv2.imread(file)
            # Check if the image exists/if it's been correctly read-in
            if not im is None:
                # Append the image, and it's type to the image list
                #image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
                #gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
                im_binary,im_canny = canny(im)
                contours = contourrr(im,im_binary)
                roi_u,roi_l,total = roi(im,im_binary,contours,ROI_number)
                train_images.append(roi_u)
                train_labels.append(im_type)
                #train_images.append(roi_l)
                #train_labels.append(im_type)
                #i_list.append((roi_u,im_type))
                #i_list.append((roi_l,im_type))
                #flat_1 = np.array(roi_u).flatten()
                #flat_2 = np.array(roi_l).flatten()
                #svm_list.append((flat_1,im_type))
                #svm_list.append((flat_2,im_type))
                ROI_number += 1
                
    print(ROI_number)
    train_images = np.array(train_images,dtype=object)
    train_labels = np.array(train_labels,dtype=object)
    return train_images,train_labels



In [6]:
def loadtest_dataset(image_dir):
    ROI_number = 1
    test_images = []
    test_labels = [] 
    i_list = []
    image_types = ["Defected","Ok"]
    
    # Iterate through each color folder
    for im_type in image_types:
        
        for file in glob.glob(os.path.join(image_dir, im_type, "*")):
            
            # Read in the image
            im = mpimg.imread(file)
            # Check if the image exists/if it's been correctly read-in
            if not im is None:
                im_binary,im_canny = canny(im)
                contours = contourrr(im,im_binary)
                roi_u,roi_l,total = roi(im,im_binary,contours,ROI_number)
                test_images.append(roi_u)
                test_labels.append(im_type)
                #test_images.append(roi_l)
                #test_labels.append(im_type)
                #i_list.append((roi_u,im_type))
                #i_list.append((roi_l,im_type))
                #flat_1 = np.array(roi_u).flatten()
                #flat_2 = np.array(roi_l).flatten()
                #svm_list.append((flat_1,im_type))
                #svm_list.append((flat_2,im_type))
                ROI_number += 1
                print(ROI_number)
    
    test_images = np.array(test_images,dtype=object)
    test_labels = np.array(test_labels,dtype=object)
    return test_images,test_labels


train_images,train_labels = loadtrain_dataset(train_dir)
test_images,test_labels = loadtest_dataset(test_dir)

9
--
342613.0
--
98299.0
29
--
316707.5
--
96080.5
12
--
337288.0
--
89245.5
4
8
--
354620.0
--
96770.0
2
23
--
316726.0
--
98399.0
3
41
--
307624.0
--
100838.0
4
19
--
311736.5
--
97579.0
5
7
--
323697.5
--
96353.5
6
16
--
312454.0
--
95899.5
7
11
--
313312.0
--
100379.0
8


In [7]:
len(train_images)
train_images.shape

(3,)

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)
print(train_labels_encoded)

[0 0 1]


In [ ]:
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded
x_train, x_test = x_train / 255.0, x_test / 255.0
plt.imshow(x_train[0])
img = x_train[0]

orb = cv2.ORB(nFeatures = 100)
kp = orb.detect(img,None)
kp, des = orb.compute(img, kp)
len(des)

In [2]:
def feature_extractor(dataset):
    x_train = dataset
    image_dataset = pd.DataFrame()
    for image in range(x_train.shape[0]):  #iterate through each file 
        #print(image)
        
        df = pd.DataFrame()  #Temporary data frame to capture information for each loop.
        #Reset dataframe to blank after each loop.
        
        input_img = x_train[image]
        img = input_img

    #Add feature extractors, e.g. edge detection, smoothing, etc. 
            
        # FEATURE 1 - Pixel values
         
        #Add pixel values to the data frame
        pixel_values = img.reshape(-1)
        df['Pixel_Value'] = pixel_values   #Pixel value itself as a feature
        #df['Image_Name'] = image   #Capture image name as we read multiple images
        
        # FEATURE 2 - Bunch of Gabor filter responses
        
        #Generate Gabor features
        #num = 1  #To count numbers up in order to give Gabor features a lable in the data frame
        #kernels = []
        #for theta in range(2):   #Define number of thetas
        #    theta = theta / 4. * np.pi
        #    for sigma in (1, 3):  #Sigma with 1 and 3
        #        lamda = np.pi/4
        #        gamma = 0.5
        #        gabor_label = 'Gabor' + str(num)  #Label Gabor columns as Gabor1, Gabor2, etc.
    #                print(gabor_label)
        #        ksize=9
        #        kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)    
        #        kernels.append(kernel)
        #        #Now filter the image and add values to a new column 
        #        fimg = cv2.filter2D(img, cv2.CV_8UC3, kernel)
        #        filtered_img = fimg.reshape(-1)
        #        df[gabor_label] = filtered_img  #Labels columns as Gabor1, Gabor2, etc.
        #        print(gabor_label, ': theta=', theta, ': sigma=', sigma, ': lamda=', lamda, ': gamma=', gamma)
        #        num += 1  #Increment for gabor column label
                
         
        #feature 3
        #hog = cv2.HOGDescriptor()
        #h = hog.compute(img)
        #df['HOG'] = h
       
        #feature4
        orb = cv2.ORB()
        kp = orb.detect(img,None)
        kp, des = orb.compute(img, kp)
        df['ORB'] = des
        
        #Append features from current image to the dataset
        image_dataset = image_dataset.append(df)
        
    return image_dataset

In [3]:
image_features = feature_extractor(x_train)

#Reshape to a vector for Random Forest / SVM training
n_features = image_features.shape[1]
image_features = np.expand_dims(image_features, axis=0)
X_for_RF = np.reshape(image_features, (x_train.shape[0], -1))  #Reshape to #images, features

NameError: name 'x_train' is not defined

In [ ]:
#Define the classifier
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

#Can also use SVM but RF is faster and may be more accurate.
#from sklearn import svm
#SVM_model = svm.SVC(decision_function_shape='ovo')  #For multiclass classification
#SVM_model.fit(X_for_RF, y_train)


# Fit the model on training data
RF_model.fit(X_for_RF, y_train) #For sklearn no one hot encoding

In [ ]:
#Predict on Test data
#Extract features from test data and reshape, just like training data
test_features = feature_extractor(x_test)
test_features = np.expand_dims(test_features, axis=0)
test_for_RF = np.reshape(test_features, (x_test.shape[0], -1))


In [ ]:
#Predict on test
test_prediction = RF_model.predict(test_for_RF)
#Inverse le transform to get original label back. 
test_prediction = le.inverse_transform(test_prediction)


In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, test_prediction))

#Print confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_labels, test_prediction)

fig, ax = plt.subplots(figsize=(6,6))         # Sample figsize in inches
sns.set(font_scale=1.6)
sns.heatmap(cm, annot=True, ax=ax)


In [ ]:
#Check results on a few random images
import random
n=random.randint(0, x_test.shape[0]-1) #Select the index of image to be loaded for testing
img = x_test[n]
plt.imshow(img)


In [ ]:
#Extract features and reshape to right dimensions
input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
input_img_features=feature_extractor(input_img)
input_img_features = np.expand_dims(input_img_features, axis=0)
input_img_for_RF = np.reshape(input_img_features, (input_img.shape[0], -1))


#Predict
img_prediction = RF_model.predict(input_img_for_RF)
img_prediction = le.inverse_transform([img_prediction])  #Reverse the label encoder to original name
print("The prediction for this image is: ", img_prediction)
print("The actual label for this image is: ", test_labels[n])

In [ ]:
#rng.shuffle(IMAGE_LIST)

#cv2.imwrite('1.jpg',selected_image)
#image = cv2.cvtColor(selected_image, cv2.COLOR_BGR2RGB)



In [ ]:
print('!')

In [ ]:
#image_index = 6
#selected_image = IMAGE_LIST[image_index][0]
#selected_label = IMAGE_LIST[image_index][1]

#print('1. The shape of the image',selected_image.shape)
#print(' 2. The image label:',selected_label)
3#print(selected_image)
#plt.imshow(selected_image)
#sele = np.array(IMAGE_LIST[:][0],dtype = object)
#sele1 = sele.flatten()

In [ ]:
print(roi1.shape)